In [1]:
#https://www.ers.usda.gov/data-products/county-level-data-sets/download-data/

In [2]:
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# for progress_apply
from tqdm import tqdm
tqdm.pandas()

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
os.chdir(r'/Users/philip.ballentine/Downloads/')

In [4]:
os.getcwd()

'/Users/philip.ballentine/Downloads'

In [5]:
poverty_df = pd.read_excel("~/Downloads/PovertyEstimates.xls")

In [6]:
poverty_df.head()

,"Poverty estimates for the U.S., States, and counties, 2018",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,"Source: U.S. Census Bureau, Small Area Income ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"For definitions of rural classifications, see ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"This table was prepared by USDA, Economic Rese...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2018,CI90LBAll_2018,CI90UBALL_2018,...,CI90UB517P_2018,MEDHHINC_2018,CI90LBINC_2018,CI90UBINC_2018,POV04_2018,CI90LB04_2018,CI90UB04_2018,PCTPOV04_2018,CI90LB04P_2018,CI90UB04P_2018
4,00000,US,United States,NaN,NaN,NaN,NaN,41852315,41619366,42085264,...,17.2,61937,61843,62031,3758704,3714862,3802546,19.5,19.3,19.7


In [7]:
columns = list(poverty_df.iloc[3])
poverty_reshaped = poverty_df.drop([0, 1,2])
poverty_reshaped.columns = columns
poverty_reshaped = poverty_reshaped.drop([3])

In [8]:
poverty_reshaped.dtypes

FIPStxt                            object
Stabr                              object
Area_name                          object
Rural-urban_Continuum_Code_2003    object
Urban_Influence_Code_2003          object
Rural-urban_Continuum_Code_2013    object
Urban_Influence_Code_2013          object
POVALL_2018                        object
CI90LBAll_2018                     object
CI90UBALL_2018                     object
PCTPOVALL_2018                     object
CI90LBALLP_2018                    object
CI90UBALLP_2018                    object
POV017_2018                        object
CI90LB017_2018                     object
CI90UB017_2018                     object
PCTPOV017_2018                     object
CI90LB017P_2018                    object
CI90UB017P_2018                    object
POV517_2018                        object
CI90LB517_2018                     object
CI90UB517_2018                     object
PCTPOV517_2018                     object
CI90LB517P_2018                   

In [9]:
poverty_reshaped_sub = poverty_reshaped[['FIPStxt','Stabr','Area_name','PCTPOVALL_2018','Urban_Influence_Code_2013','MEDHHINC_2018']]

In [10]:
poverty_reshaped_sub.rename(columns={'FIPStxt':'County_FIPS','Stabr':'StateCD','PCTPOVALL_2018':'Poverty_PCT_2018','Urban_Influence_Code_2013':'UrbanCD','MEDHHINC_2018':'Median_Income_2018'},inplace=True)

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
poverty_reshaped_sub.UrbanCD.fillna('UNK',inplace=True)

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [12]:
poverty_reshaped_sub.describe()

,County_FIPS,StateCD,Area_name,Poverty_PCT_2018,UrbanCD,Median_Income_2018
count,3193,3193,3193,3193.0,3193,3193
unique,3193,52,1927,317.0,13,3081
top,18003,TX,Washington County,13.8,2,49356
freq,1,255,30,33.0,733,3


## Import Education Data

In [13]:
education_df = pd.read_excel("~/Downloads/Education.xls")

In [14]:
education_df

,County_FIPS,StateCD,Name,HS_DIPLOMA_ONLY_2018,SOME_COLLEGE_2018,BACHELORS_PLUS_2018
0,0,US,United States,27.1,29.0,31.5
1,1000,AL,Alabama,30.9,29.9,24.9
2,1001,AL,Autauga County,32.6,28.4,27.7
3,1003,AL,Baldwin County,27.6,31.3,31.3
4,1005,AL,Barbour County,35.7,25.1,12.2
...,...,...,...,...,...,...
3278,72145,PR,Vega Baja Municipio,23.0,25.2,20.7
3279,72147,PR,Vieques Municipio,37.3,17.0,20.6
3280,72149,PR,Villalba Municipio,36.7,18.7,20.1
3281,72151,PR,Yabucoa Municipio,25.0,26.3,18.6


In [15]:
poverty_reshaped_sub.County_FIPS = poverty_reshaped_sub.County_FIPS.astype("str")

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
import addfips
def lookup_location(locationstring):
    af = addfips.AddFIPS()
    try:
        namelist = locationstring.split('|')
        statename = namelist[1]
        countyname = namelist[0]
        fipscode = af.get_county_fips(countyname, state=statename)
        return str(fipscode)
    except:
        return '00000'

In [17]:
lookup_location("Broomfield|CO")
# check to see if leading zeroes preserved

'08014'

## Create Joinkey for join

In [18]:
education_df['locationstring'] = education_df['Name']+'|'+education_df['StateCD']

#create keycol
    

In [19]:
education_df['FIPS_LookedUp'] = education_df['locationstring'].progress_apply(lookup_location)

# look up the FIPS

100%|██████████| 3283/3283 [01:08<00:00, 48.20it/s]


In [20]:
education_df['FIPS_Coalesce'] = education_df.FIPS_LookedUp.combine_first(education_df.County_FIPS)

# coalesce FIPS

In [21]:
poverty_reshaped_sub['locationstring'] = poverty_reshaped_sub['Area_name']+'|'+poverty_reshaped_sub['StateCD']

#create keycol

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
poverty_reshaped_sub['FIPS_LookedUp'] = poverty_reshaped_sub['locationstring'].apply(lookup_location)

# look up the FIPS

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
poverty_reshaped_sub['FIPS_Coalesce'] = poverty_reshaped_sub.County_FIPS.combine_first(poverty_reshaped_sub.FIPS_LookedUp)

# coalesce FIPS

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
poverty_reshaped_sub['FIPS_Coalesce'] = poverty_reshaped_sub['FIPS_Coalesce'].astype("str")


/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
poverty_reshaped_sub[poverty_reshaped_sub['StateCD'] == "CO"]

# check CO for the leading zero issue

,County_FIPS,StateCD,Area_name,Poverty_PCT_2018,UrbanCD,Median_Income_2018,locationstring,FIPS_LookedUp,FIPS_Coalesce
254,08000,CO,Colorado,9.7,UNK,71949,Colorado|CO,None,08000
255,08001,CO,Adams County,9.2,1,70199,Adams County|CO,08001,08001
256,08003,CO,Alamosa County,19.1,11,42897,Alamosa County|CO,08003,08003
257,08005,CO,Arapahoe County,8.1,1,76665,Arapahoe County|CO,08005,08005
258,08007,CO,Archuleta County,11.2,10,53337,Archuleta County|CO,08007,08007
...,...,...,...,...,...,...,...,...,...
314,08117,CO,Summit County,6.7,8,80078,Summit County|CO,08117,08117
315,08119,CO,Teller County,7.4,2,64850,Teller County|CO,08119,08119
316,08121,CO,Washington County,13.7,10,47869,Washington County|CO,08121,08121
317,08123,CO,Weld County,10.5,2,75629,Weld County|CO,08123,08123


In [26]:
education_df['FIPS_Coalesce'] = education_df['FIPS_Coalesce'].astype("str")

In [27]:
education_df[education_df['StateCD'] == "CO"]

# check CO for the leading zero issue

,County_FIPS,StateCD,Name,HS_DIPLOMA_ONLY_2018,SOME_COLLEGE_2018,BACHELORS_PLUS_2018,locationstring,FIPS_LookedUp,FIPS_Coalesce
257,8000,CO,Colorado,21.4,29.8,40.1,Colorado|CO,None,None
258,8001,CO,Adams County,28.4,31.0,23.6,Adams County|CO,08001,08001
259,8003,CO,Alamosa County,27.3,33.1,26.1,Alamosa County|CO,08003,08003
260,8005,CO,Arapahoe County,20.7,29.5,42.2,Arapahoe County|CO,08005,08005
261,8007,CO,Archuleta County,21.5,31.7,38.9,Archuleta County|CO,08007,08007
...,...,...,...,...,...,...,...,...,...
317,8117,CO,Summit County,19.0,23.3,51.0,Summit County|CO,08117,08117
318,8119,CO,Teller County,25.5,36.2,34.0,Teller County|CO,08119,08119
319,8121,CO,Washington County,32.0,44.3,16.2,Washington County|CO,08121,08121
320,8123,CO,Weld County,27.5,33.8,26.7,Weld County|CO,08123,08123


## Merge Education and Poverty Datasets

In [28]:
e_p_df = education_df.merge(poverty_reshaped_sub, on="FIPS_Coalesce", how="outer")

# 

In [29]:
e_p_df.columns

Index(['County_FIPS_x', 'StateCD_x', 'Name', 'HS_DIPLOMA_ONLY_2018',
       'SOME_COLLEGE_2018', 'BACHELORS_PLUS_2018', 'locationstring_x',
       'FIPS_LookedUp_x', 'FIPS_Coalesce', 'County_FIPS_y', 'StateCD_y',
       'Area_name', 'Poverty_PCT_2018', 'UrbanCD', 'Median_Income_2018',
       'locationstring_y', 'FIPS_LookedUp_y'],
      dtype='object')

In [30]:
e = e_p_df 

In [31]:
problem1 = e[e['HS_DIPLOMA_ONLY_2018'].isna() ==True] # 334


In [32]:
problem = e[e['Median_Income_2018'].isna() ==True] # 414

In [33]:
problem.StateCD_x.value_counts()

PR    79
AK     8
VA     3
MT     2
IL     2
IN     1
DE     1
US     1
TN     1
VT     1
NM     1
CO     1
MO     1
HI     1
MN     1
CT     1
MI     1
WI     1
RI     1
MS     1
LA     1
PA     1
OR     1
NH     1
AZ     1
NC     1
MA     1
TX     1
KS     1
ME     1
GA     1
WY     1
KY     1
OH     1
FL     1
CA     1
NJ     1
NE     1
ND     1
WV     1
WA     1
AL     1
NV     1
SC     1
SD     1
MD     1
Name: StateCD_x, dtype: int64

In [34]:
problem1.StateCD_y.value_counts()

IL    2
CT    1
MS    1
LA    1
PA    1
OK    1
MI    1
NC    1
KS    1
DC    1
AZ    1
VT    1
WY    1
MT    1
ND    1
AR    1
TN    1
NY    1
NM    1
CO    1
IN    1
HI    1
RI    1
WI    1
OR    1
FL    1
KY    1
TX    1
MA    1
AL    1
ME    1
GA    1
SC    1
MN    1
UT    1
ID    1
VA    1
OH    1
US    1
CA    1
SD    1
WV    1
NE    1
IA    1
WA    1
MO    1
NV    1
MD    1
NJ    1
DE    1
NH    1
AK    1
Name: StateCD_y, dtype: int64

In [35]:
e_nona = e.dropna()

In [36]:
e_nona.columns

Index(['County_FIPS_x', 'StateCD_x', 'Name', 'HS_DIPLOMA_ONLY_2018',
       'SOME_COLLEGE_2018', 'BACHELORS_PLUS_2018', 'locationstring_x',
       'FIPS_LookedUp_x', 'FIPS_Coalesce', 'County_FIPS_y', 'StateCD_y',
       'Area_name', 'Poverty_PCT_2018', 'UrbanCD', 'Median_Income_2018',
       'locationstring_y', 'FIPS_LookedUp_y'],
      dtype='object')

## FINDING DUPLICATES (STATES MASQUERADING AS COUNTIES)

In [37]:
e_vc = pd.DataFrame(e_nona.FIPS_Coalesce.value_counts()).reset_index()

In [38]:
dup = e_vc[e_vc.FIPS_Coalesce > 1]
dup

,index,FIPS_Coalesce
0,15001,2
1,16049,2
2,05001,2
3,40109,2
4,19095,2
5,49049,2
6,11001,2
7,36061,2


In [39]:
list_dups = list(dup['index'])
#list_dups.remove('05001')
list_dups

['15001', '16049', '05001', '40109', '19095', '49049', '11001', '36061']

In [40]:
# Take rows without any dups
clean_values = e_nona[e_nona.FIPS_LookedUp_x.isin(list_dups) == False]

In [41]:
# Take rows with dups
dup_values = e_nona[e_nona.FIPS_Coalesce.isin(list_dups) == True]

In [42]:
dup_values.sort_values(by="StateCD_x")

,County_FIPS_x,StateCD_x,Name,HS_DIPLOMA_ONLY_2018,SOME_COLLEGE_2018,BACHELORS_PLUS_2018,locationstring_x,FIPS_LookedUp_x,FIPS_Coalesce,County_FIPS_y,StateCD_y,Area_name,Poverty_PCT_2018,UrbanCD,Median_Income_2018,locationstring_y,FIPS_LookedUp_y
166,5000.0,AR,Arkansas,34.1,29.5,22.6,Arkansas|AR,05001,05001,05001,AR,Arkansas County,17.9,6,43887,Arkansas County|AR,05001
167,5001.0,AR,Arkansas County,38.3,29.2,15.5,Arkansas County|AR,05001,05001,05001,AR,Arkansas County,17.9,6,43887,Arkansas County|AR,05001
375,11000.0,DC,District of Columbia,17.2,15.9,57.6,District of Columbia|DC,11001,11001,11001,DC,District of Columbia,16.1,1,82533,District of Columbia|DC,11001
376,11001.0,DC,District of Columbia,17.2,15.9,57.6,District of Columbia|DC,11001,11001,11001,DC,District of Columbia,16.1,1,82533,District of Columbia|DC,11001
603,15000.0,HI,Hawaii,27.7,31.7,32.5,Hawaii|HI,15001,15001,15001,HI,Hawaii County,15.6,8,56383,Hawaii County|HI,15001
604,15001.0,HI,Hawaii County,30.4,32.5,29.5,Hawaii County|HI,15001,15001,15001,HI,Hawaii County,15.6,8,56383,Hawaii County|HI,15001
847,19000.0,IA,Iowa,31.1,32.6,28.2,Iowa|IA,19095,19095,19095,IA,Iowa County,7.4,6,64188,Iowa County|IA,19095
848,19095.0,IA,Iowa County,37.7,37.0,19.1,Iowa County|IA,19095,19095,19095,IA,Iowa County,7.4,6,64188,Iowa County|IA,19095
609,16000.0,ID,Idaho,27.7,35.9,26.9,Idaho|ID,16049,16049,16049,ID,Idaho County,16.2,6,40569,Idaho County|ID,16049
610,16049.0,ID,Idaho County,35.1,35.3,20.2,Idaho County|ID,16049,16049,16049,ID,Idaho County,16.2,6,40569,Idaho County|ID,16049


In [43]:
def remove_non_county(x):
    if "County" not in x:
        return "Statelevel"
    else:
        return "Countylevel"
    

In [44]:
# Remove the states masquerading as counties
dup_values_countiesonly = dup_values[dup_values['locationstring_x'].apply(remove_non_county) == "Countylevel"]

In [45]:
dup_values_countiesonly

,County_FIPS_x,StateCD_x,Name,HS_DIPLOMA_ONLY_2018,SOME_COLLEGE_2018,BACHELORS_PLUS_2018,locationstring_x,FIPS_LookedUp_x,FIPS_Coalesce,County_FIPS_y,StateCD_y,Area_name,Poverty_PCT_2018,UrbanCD,Median_Income_2018,locationstring_y,FIPS_LookedUp_y
167,5001.0,AR,Arkansas County,38.3,29.2,15.5,Arkansas County|AR,05001,05001,05001,AR,Arkansas County,17.9,6,43887,Arkansas County|AR,05001
604,15001.0,HI,Hawaii County,30.4,32.5,29.5,Hawaii County|HI,15001,15001,15001,HI,Hawaii County,15.6,8,56383,Hawaii County|HI,15001
610,16049.0,ID,Idaho County,35.1,35.3,20.2,Idaho County|ID,16049,16049,16049,ID,Idaho County,16.2,6,40569,Idaho County|ID,16049
848,19095.0,IA,Iowa County,37.7,37.0,19.1,Iowa County|IA,19095,19095,19095,IA,Iowa County,7.4,6,64188,Iowa County|IA,19095
1888,36061.0,NY,New York County,12.4,13.7,60.8,New York County|NY,36061,36061,36061,NY,New York County,15.6,1,84610,New York County|NY,36061
2192,40109.0,OK,Oklahoma County,25.3,30.1,31.8,Oklahoma County|OK,40109,40109,40109,OK,Oklahoma County,16.9,1,52346,Oklahoma County|OK,40109
2839,49049.0,UT,Utah County,16.3,37.8,40.1,Utah County|UT,49049,49049,49049,UT,Utah County,9.4,2,75296,Utah County|UT,49049


In [46]:
# Replace the looked up FIPS with the original FIPS for dups ,fixes all but AR which has leading zero issue
dup_values['FIPS_Coalesce'] = dup_values['FIPS_LookedUp_x']

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
# Recombine
cleaned_table = pd.concat([clean_values,dup_values_countiesonly])

In [48]:
# Check for dups again
dupcheck = pd.DataFrame(cleaned_table.FIPS_Coalesce.value_counts()).reset_index()

In [49]:
dupcheck[dupcheck['FIPS_Coalesce']>1]

,index,FIPS_Coalesce


In [50]:
cleaned_table['len'] = cleaned_table['FIPS_Coalesce'].str.len()

In [51]:
cleaned_table[cleaned_table['FIPS_Coalesce'].str.len() != 5]

,County_FIPS_x,StateCD_x,Name,HS_DIPLOMA_ONLY_2018,SOME_COLLEGE_2018,BACHELORS_PLUS_2018,locationstring_x,FIPS_LookedUp_x,FIPS_Coalesce,County_FIPS_y,StateCD_y,Area_name,Poverty_PCT_2018,UrbanCD,Median_Income_2018,locationstring_y,FIPS_LookedUp_y,len


In [52]:
#cleaned_table['FIPS_Coalesce'] = cleaned_table['FIPS_Coalesce'].str.replace(r'\b5001\b','05001')

In [53]:
#pd.Series('5001').str.replace(r'\b5001\b','05001')

In [54]:
# Check Arkansas for leading zero
cleaned_table[cleaned_table.FIPS_Coalesce == '05001']

,County_FIPS_x,StateCD_x,Name,HS_DIPLOMA_ONLY_2018,SOME_COLLEGE_2018,BACHELORS_PLUS_2018,locationstring_x,FIPS_LookedUp_x,FIPS_Coalesce,County_FIPS_y,StateCD_y,Area_name,Poverty_PCT_2018,UrbanCD,Median_Income_2018,locationstring_y,FIPS_LookedUp_y,len
167,5001.0,AR,Arkansas County,38.3,29.2,15.5,Arkansas County|AR,05001,05001,05001,AR,Arkansas County,17.9,6,43887,Arkansas County|AR,05001,5


In [55]:
colslist = ['FIPS_Coalesce','StateCD_y','Name','HS_DIPLOMA_ONLY_2018',
 'SOME_COLLEGE_2018','BACHELORS_PLUS_2018','Poverty_PCT_2018',
 'UrbanCD','Median_Income_2018']

In [56]:
# subset to just these columns 
cleaned_table_sub = cleaned_table[colslist]

In [57]:
cleaned_table_sub.rename(columns={'FIPS_Coalesce':'County_FIPS','StateCD_y':'StateCD'}, inplace=True)

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [58]:
cleaned_table_sub.reset_index(inplace=True, drop=True)

In [59]:
cleaned_table_sub.dtypes

County_FIPS              object
StateCD                  object
Name                     object
HS_DIPLOMA_ONLY_2018    float64
SOME_COLLEGE_2018       float64
BACHELORS_PLUS_2018     float64
Poverty_PCT_2018         object
UrbanCD                  object
Median_Income_2018       object
dtype: object

In [60]:
cleaned_table_sub['Median_Income_2018'] = cleaned_table_sub['Median_Income_2018'].astype("float64")

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
cleaned_table_sub['Poverty_PCT_2018'] = cleaned_table_sub['Poverty_PCT_2018'].astype("float64")

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
cleaned_table_sub.columns

Index(['County_FIPS', 'StateCD', 'Name', 'HS_DIPLOMA_ONLY_2018',
       'SOME_COLLEGE_2018', 'BACHELORS_PLUS_2018', 'Poverty_PCT_2018',
       'UrbanCD', 'Median_Income_2018'],
      dtype='object')

## Get the Items from Census by FIPS code

In [63]:
# https://pypi.org/project/census/


from census import Census
from us import states

# Test
c = Census("3849978012ed677b69607fe6ce07f38d261a4244")
c.acs5.get(('NAME', 'B25034_010E'),
          {'for': 'state:{}'.format(states.MD.fips)})

[{'NAME': 'Maryland', 'B25034_010E': 129556.0, 'state': '24'}]

## Use Census API for Population Data

In [64]:
# census api : https://api.census.gov/data/2016/acs/acs5/groups/B01003/
# population :  'B01003_001E'

def look_up_census_pop_from_fips(county_fips):
    item_id = 'B01003_001E' # Total Population
    try:
        sf = county_fips[0:2]
        cf = county_fips[2:6]
        population_info = c.acs5.state_county((item_id), state_fips='{sf}'.format(sf=sf), county_fips='{cf}'.format(cf=cf))
        #return (sf, cf) 
        return population_info[0][item_id] 
    except:
        return np.NaN

In [65]:
cleaned_table_sub['Population_ACS'] = cleaned_table_sub['County_FIPS'].progress_apply(look_up_census_pop_from_fips)

100%|██████████| 3139/3139 [39:44<00:00,  1.32it/s]  
/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
cleaned_table_sub

,County_FIPS,StateCD,Name,HS_DIPLOMA_ONLY_2018,SOME_COLLEGE_2018,BACHELORS_PLUS_2018,Poverty_PCT_2018,UrbanCD,Median_Income_2018,Population_ACS
0,01001,AL,Autauga County,32.6,28.4,27.7,13.8,2,59338.0,55036.0
1,01003,AL,Baldwin County,27.6,31.3,31.3,9.8,2,57588.0,203360.0
2,01005,AL,Barbour County,35.7,25.1,12.2,30.9,6,34382.0,26201.0
3,01007,AL,Bibb County,47.3,24.4,11.5,21.8,1,46064.0,22580.0
4,01009,AL,Blount County,34.0,33.5,12.6,13.2,1,50412.0,57667.0
...,...,...,...,...,...,...,...,...,...,...
3134,16049,ID,Idaho County,35.1,35.3,20.2,16.2,6,40569.0,16275.0
3135,19095,IA,Iowa County,37.7,37.0,19.1,7.4,6,64188.0,16231.0
3136,36061,NY,New York County,12.4,13.7,60.8,15.6,1,84610.0,1653877.0
3137,40109,OK,Oklahoma County,25.3,30.1,31.8,16.9,1,52346.0,774203.0


In [67]:
cleaned_table_sub['Population_ACS'].isna().value_counts()

False    3139
Name: Population_ACS, dtype: int64

In [68]:
# census api : https://api.census.gov/data/2016/acs/acs5/groups/B01003/
# https://api.census.gov/data/2016/acs/acs5/variables.html
# population :  'B01003_001E'

def look_up_census_black_pop_from_fips(county_fips):
    item_id = 'B02001_003E' # Total Black population
    try:
        sf = county_fips[0:2]
        cf = county_fips[2:6]
        population_info = c.acs5.state_county((item_id), state_fips='{sf}'.format(sf=sf), county_fips='{cf}'.format(cf=cf))
        #return (sf, cf) 
        return population_info[0][item_id] 
    except:
        return np.NaN

In [ ]:
cleaned_table_sub['Population_Black_ACS'] = cleaned_table_sub['County_FIPS'].progress_apply(look_up_census_black_pop_from_fips)

  9%|▊         | 271/3139 [03:13<35:49,  1.33it/s]

In [ ]:
# https://api.census.gov/data/2016/acs/acs5/variables/B03001_003E.json
# https://api.census.gov/data/2016/acs/acs5/variables.html

def look_up_census_hisp_pop_from_fips(county_fips):
    item_id = 'B03001_003E' # Total Black population
    try:
        sf = county_fips[0:2]
        cf = county_fips[2:6]
        population_info = c.acs5.state_county((item_id), state_fips='{sf}'.format(sf=sf), county_fips='{cf}'.format(cf=cf))
        #return (sf, cf) 
        return population_info[0][item_id] 
    except:
        return np.NaN

In [ ]:
cleaned_table_sub['Population_Hisp_ACS'] = cleaned_table_sub['County_FIPS'].progress_apply(look_up_census_hisp_pop_from_fips)

In [ ]:
# https://api.census.gov/data/2016/acs/acs5/variables/B03001_003E.json
# https://api.census.gov/data/2016/acs/acs5/variables.html

def look_up_census_native_pop_from_fips(county_fips):
    item_id = 'B01001C_001E' # Total American Indian / Alaska Native population
    try:
        sf = county_fips[0:2]
        cf = county_fips[2:6]
        population_info = c.acs5.state_county((item_id), state_fips='{sf}'.format(sf=sf), county_fips='{cf}'.format(cf=cf))
        #return (sf, cf) 
        return population_info[0][item_id] 
    except:
        return np.NaN

In [ ]:
cleaned_table_sub['Population_Native_ACS'] = cleaned_table_sub['County_FIPS'].progress_apply(look_up_census_native_pop_from_fips)

In [ ]:
cleaned_table_sub.columns

In [ ]:
cleaned_table_sub['PCT_Black_ACS'] = 100*(cleaned_table_sub['Population_Black_ACS']/cleaned_table_sub['Population_ACS'])

In [ ]:
cleaned_table_sub['PCT_Hisp_ACS'] = 100*(cleaned_table_sub['Population_Hisp_ACS']/cleaned_table_sub['Population_ACS'])

In [ ]:
cleaned_table_sub['PCT_Native_ACS'] = 100*(cleaned_table_sub['Population_Native_ACS']/cleaned_table_sub['Population_ACS'])

## Export to File

In [ ]:
import os
os.chdir(r'/Users/philip.ballentine/Documents/Scripts')
path = str(os.getcwd())
filename = "counties_dataset_full_20201121_3.csv"
cleaned_table_sub.to_csv(filename) # un-comment to actually create new file
print("{filename} has been created in {path}".format(filename=filename, path=path))

## Get Descriptions for the Urban Codes 

In [ ]:
#https://www.ers.usda.gov/data-products/urban-influence-codes/documentation.aspx
rural_urban = pd.read_html("https://www.ers.usda.gov/data-products/urban-influence-codes/documentation.aspx")
rural_urban_frame = rural_urban[0]
goodlist = [str(i) for i in range(0,15)] # filter out the descriptor fields 
rural_urban_frame = rural_urban_frame[rural_urban_frame.Code.isin(goodlist)]
rural_urban_frame = rural_urban_frame[['Code','Description']]
rural_urban_frame.columns = ['UrbanCD','UrbanCD_Description']

In [ ]:
cleaned_table_sub

In [ ]:
cleaned_table_sub['UrbanCD'] = cleaned_table_sub['UrbanCD'].astype("str")

In [ ]:
cleaned_table_sub_rural_urban = cleaned_table_sub.merge(rural_urban_frame, on="UrbanCD")

In [ ]:
rural_urban_frame

In [ ]:
cleaned_table_sub_rural_urban.UrbanCD_Description.value_counts()

In [ ]:
def discern_metro(x):
    if float(x) >= 1 and float(x) <=12:
        x = int(x)
        if x == 1:
            return 'Large_metro' # over 1 mil 
        if x == 2:
            return 'Smaller_metro' # under 1 mil
        if x in [3,5,8]:
            return 'Micropolitan'
        if x in [4,6,7,10]:
            return 'Noncore_adjacent' # Non-metro areas but near metro area, some commuting
        if x in [11,12]:
            return 'Noncore_remote'
    else:
        return np.NaN

In [ ]:
cleaned_table_sub_rural_urban['Metro'] = cleaned_table_sub_rural_urban['UrbanCD'].progress_apply(discern_metro)

In [ ]:
cleaned_table_sub_rural_urban.Metro.value_counts()

In [ ]:
samp = cleaned_table_sub_rural_urban.sample(frac=.05)#[.Metro.value_counts()]

In [ ]:
samp[samp.Metro=="Large_metro"]